# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Updating-dmLL" data-toc-modified-id="Updating-dmLL-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Updating dmLL</a></div><div class="lev2 toc-item"><a href="#Optimize-grad_stack" data-toc-modified-id="Optimize-grad_stack-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Optimize grad_stack</a></div><div class="lev2 toc-item"><a href="#Optimize-update_mll_and_dmll" data-toc-modified-id="Optimize-update_mll_and_dmll-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Optimize update_mll_and_dmll</a></div><div class="lev1 toc-item"><a href="#Update-mLL" data-toc-modified-id="Update-mLL-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Update mLL</a></div>

In [1]:
using GaussianProcesses

In [2]:
import PyPlot; plt=PyPlot
using LaTeXStrings
plt.rc("figure", dpi=300.0)
# plt.rc("figure", figsize=(6,4))
plt.rc("figure", autolayout=true)
plt.rc("savefig", dpi=300.0)
plt.rc("text", usetex=true)
plt.rc("font", family="serif")
plt.rc("font", serif="Palatino")
using TimeSeries
using DataFrames
using GaussianProcesses
using GaussianProcesses: num_params, set_params!, get_params, grad_stack, grad_stack!, update_mll!, update_mll_and_dmll!, dKdθi!
import Optim
;

In [3]:
station_data = readtable("data2015/725450.14990.processed.2015.2015.csv", 
    header=false,
    names=[:year, :month, :day, :hour, :min, :seconds, :temp])
station_data = station_data[!isnan(station_data[:temp]),:]    
station_ts = DateTime[DateTime(r[:year], r[:month], r[:day], r[:hour], r[:min], r[:seconds]) for r in eachrow(station_data)]
station_ts[1:10]

10-element Array{DateTime,1}:
 2015-01-01T00:52:00
 2015-01-01T01:52:00
 2015-01-01T02:52:00
 2015-01-01T03:52:00
 2015-01-01T04:52:00
 2015-01-01T05:52:00
 2015-01-01T06:52:00
 2015-01-01T07:52:00
 2015-01-01T08:52:00
 2015-01-01T09:52:00

In [4]:
k0 = Noise(0.0)
k1 = Periodic(log(0.5), log(√(10.0)), log(24.0))
k2 = RQIso(log(0.1*24), log(√10.0), 3.0) # short term
k3 = RQIso(log(3.0*24), log(√20.0), 3.0)
k4 = RQIso(log(10.0*24), log(√20.0), 3.0)
k5 = SE(log(200.0*24), log(√80.0))
_k=k0+k1+k2+k3+k4+k5
_k_nonoise=k1+k2+k3+k4+k5
_logNoise=k0.lσ
hyp=[-0.359207,0.36883,1.45171,3.17741,2.0342,1.17682,2.88579,4.28659,1.49533,-0.16544,6.81425,1.3045,3.49925,7.34764,2.16028]
GaussianProcesses.set_params!(_k, hyp)

In [5]:
ms_per_hour = 1e3*3600
ts_vec = convert(Vector{Float64}, station_ts.-station_ts[1]) / ms_per_hour
ts_arr = reshape(ts_vec, (1,length(ts_vec)));

In [6]:
mean_func = MeanConst(mean(station_data[:temp]))
temp_vec = convert(Vector{Float64}, station_data[:temp])
@time gp_fit = GP(ts_arr[1:4000], temp_vec[1:4000], mean_func, _k_nonoise, -9.0);
gp_small = GP(ts_arr[1:100], temp_vec[1:100], mean_func, _k_nonoise, -9.0);

  6.088243 seconds (2.16 M allocations: 1.277 GB, 13.61% gc time)


In [7]:
size(temp_vec)

(11118,)

# Updating dmLL

In [8]:
GaussianProcesses.update_mll_and_dmll!(gp_small);
@time GaussianProcesses.update_mll_and_dmll!(gp_fit);
prev_dmLL=gp_small.dmLL

 67.387235 seconds (168.25 M allocations: 16.217 GB, 12.73% gc time)


16-element Array{Float64,1}:
      2.05225e8
     -4.10579  
     -1.83856e7
      2.26609e6
      7.07995e8
     -1.87513e7
      1.76462e6
     -2.21872e6
 -26706.3      
  34353.6      
   1796.55     
   -188.466    
    171.744    
   -132.615    
   -359.407    
   1760.4      

## Optimize grad_stack

In [9]:
function grad_stack!(stack::AbstractArray, rq::RQIso, X::Matrix{Float64}, data::GaussianProcesses.IsotropicData)
    nobsv = size(X,2)
    R = GaussianProcesses.distance(rq, X, data)
    
    for i in 1:nobsv, j in 1:i
        # Check these derivatives!
        @inbounds stack[i,j,1] = rq.σ2*((R[i,j])/rq.ℓ2)*(1.0+(R[i,j])/(2*rq.α*rq.ℓ2))^(-rq.α-1.0)  # dK_d(log ℓ)dK_dℓ
        @inbounds stack[i,j,2] = 2.0*rq.σ2*(1+(R[i,j])/(2*rq.α*rq.ℓ2))^(-rq.α) # dK_d(log σ)

        part = (1.0+R[i,j]/(2*rq.α*rq.ℓ2))
        @inbounds stack[i,j,3] = rq.σ2*part^(-rq.α)*((R[i,j])/(2*rq.ℓ2*part)-rq.α*log(part))*rq.α  # dK_d(log α)
        
        @inbounds stack[j,i,1] = stack[i,j,1]
        @inbounds stack[j,i,3] = stack[i,j,3]
        @inbounds stack[j,i,2] = stack[i,j,2]
    end
    return stack
end

LoadError: LoadError: error in method definition: function GaussianProcesses.grad_stack! must be explicitly imported to be extended
while loading In[9], in expression starting on line 1

In [12]:
_rq = gp_fit.k.kerns[3]
Ksmall = Array(Float64, gp_small.nobsv, gp_small.nobsv)
Kgrad = Array(Float64, gp_fit.nobsv, gp_fit.nobsv)
dKdθi!(Ksmall, gp_small.k, gp_small.X, gp_small.data,5)
# prev_grad_stack = Ksmall;
@time dKdθi!(Kgrad, gp_fit.k, gp_fit.X, gp_fit.data, 5);

  0.617789 seconds (22 allocations: 912 bytes)


In [29]:
gp_fit.k.kerns

5-element Array{GaussianProcesses.Kernel,1}:
 Type: GaussianProcesses.Periodic, Params: [0.36883,1.45171,3.17741]

 Type: GaussianProcesses.RQIso, Params: [2.0342,1.17682,2.88579]
    
 Type: GaussianProcesses.RQIso, Params: [4.28659,1.49533,-0.16544]
  
 Type: GaussianProcesses.RQIso, Params: [6.81425,1.3045,3.49925]
    
 Type: GaussianProcesses.SEIso, Params: [7.34764,2.16028]
           

In [80]:
function GaussianProcesses.dKdθi!(Kgrad::AbstractMatrix,  pe::Periodic, X::Matrix{Float64}, data::GaussianProcesses.IsotropicData, iparam::Int)
    1 <= iparam <= num_params(pe) || throw(ArgumentError, "iparam must be between 1 and %d", num_params(pe))
    d, nobsv = size(X)
    R = GaussianProcesses.distance!(pe, X, data)
    
    if iparam==1
        @inbounds for j in 1:nobsv
            @simd for i in 1:j        
                Kgrad[i,j] = 4.0*pe.σ2*(sin(π*R[i,j]/pe.p)^2/pe.ℓ2)*exp(-2/pe.ℓ2*sin(π*R[i,j]/pe.p)^2)  # dK_dlogℓ
                Kgrad[j,i] = Kgrad[i,j]
            end
        end
    elseif iparam==2
        @inbounds for j in 1:nobsv
            @simd for i in 1:j        
                Kgrad[i,j] = 2.0*pe.σ2*exp(-2/pe.ℓ2*sin(π*R[i,j]/pe.p)^2)        # dK_dlogσ
                Kgrad[j,i] = Kgrad[i,j]
            end
        end
    elseif iparam==3 
        @inbounds for j in 1:nobsv
            @simd for i in 1:j  
                Kgrad[i,j] = 4.0/pe.ℓ2*pe.σ2*(π*R[i,j]/pe.p)*sin(π*R[i,j]/pe.p)*cos(π*R[i,j]/pe.p)*exp(-2/pe.ℓ2*sin(π*R[i,j]/pe.p)^2)    # dK_dlogp
                Kgrad[j,i] = Kgrad[i,j]
            end
        end
    end
    return Kgrad
end

In [98]:
function dKdθi3!(Kgrad::Matrix{Float64},  pe::Periodic, X::Matrix{Float64}, data::GaussianProcesses.IsotropicData, iparam::Int)
    1 <= iparam <= num_params(pe) || throw(ArgumentError, "iparam must be between 1 and %d", num_params(pe))
    d, nobsv = size(X)
    R = GaussianProcesses.distance!(pe, X, data)
    @inbounds for j in 1:nobsv
        @simd for i in 1:j
            phase=π*R[i,j]/pe.p
            Kgrad[i,j] = 4.0/pe.ℓ2*pe.σ2*(phase)*sin(phase)*cos(phase)*exp(-2/pe.ℓ2*sin(phase)^2.0)    # dK_dlogp
            Kgrad[j,i] = Kgrad[i,j]
        end
    end
    return Kgrad
end

dKdθi3! (generic function with 2 methods)

In [79]:
@time dKdθi!(Kgrad, 
    gp_fit.k.kerns[1], 
    gp_fit.X, 
    gp_fit.data.datadict[Symbol("IsotropicData_Distances.Euclidean()")],
    3);

  0.702153 seconds (4 allocations: 160 bytes)


In [101]:
@time dKdθi3!(Kgrad, 
    gp_fit.k.kerns[1], 
    gp_fit.X, 
    gp_fit.data.datadict[Symbol("IsotropicData_Distances.Euclidean()")],
    3);

  1.309343 seconds (4 allocations: 160 bytes)


In [109]:
function dKdθi3!(Kgrad::Matrix{Float64},  pe::Periodic, X::Matrix{Float64}, data::GaussianProcesses.IsotropicData, iparam::Int)
    1 <= iparam <= num_params(pe) || throw(ArgumentError, "iparam must be between 1 and %d", num_params(pe))
    d, nobsv = size(X)
    R = GaussianProcesses.distance!(pe, X, data)
    K = 0.0
    @inbounds for j in 1:nobsv
        @simd for i in 1:j
            phase=π*R[i,j]/pe.p
            K += 4.0/pe.ℓ2*pe.σ2*(phase)*sin(phase)*cos(phase)*exp(-2/pe.ℓ2*sin(phase)^2.0)    # dK_dlogp
        end
    end
    return Kgrad
end

dKdθi3! (generic function with 2 methods)

In [110]:
@time dKdθi3!(Kgrad, 
    gp_fit.k.kerns[1], 
    gp_fit.X, 
    gp_fit.data.datadict[Symbol("IsotropicData_Distances.Euclidean()")],
    3);

  0.637696 seconds (9.81 k allocations: 400.853 KB)


In [42]:
@code_warntype dKdθi!(Kgrad, 
    gp_fit.k.kerns[1], 
    gp_fit.X, 
    gp_fit.data.datadict[Symbol("IsotropicData_Distances.Euclidean()")],
    3);

Variables:
  #self#::GaussianProcesses.#dKdθi!
  Kgrad::Array{Float64,2}
  pe::GaussianProcesses.Periodic
  X::Array{Float64,2}
  data::GaussianProcesses.IsotropicData
  iparam::Int64
  d::Int64
  nobsv::Int64
  #temp#@_9::Int64
  R::Array{Float64,2}
  #temp#@_11::Int64
  j@_12::Int64
  r#309::UnitRange{Int64}
  #temp#@_14::Int64
  i#310::Int64
  n#311::Int64
  i@_17::Int64
  i#312::Int64
  i@_19::Int64
  #temp#@_20::Int64
  j@_21::Int64
  r#305::UnitRange{Int64}
  #temp#@_23::Int64
  i#306::Int64
  n#307::Int64
  i@_26::Int64
  i#308::Int64
  i@_28::Int64
  #temp#@_29::Int64
  j@_30::Int64
  r#301::UnitRange{Int64}
  #temp#@_32::Int64
  i#302::Int64
  n#303::Int64
  i@_35::Int64
  i#304::Int64
  i@_37::Int64
  #temp#@_38::Bool
  ret@_39::Int64
  ret@_40::Int64
  ret@_41::Int64
  ret@_42::Int64
  ret@_43::Int64
  ret@_44::Int64

Body:
  begin 
      NewvarNode(:(d::Int64))
      NewvarNode(:(nobsv::Int64))
      NewvarNode(:(#temp#@_9::Int64))
      NewvarNode(:(R::Array{Float64,2}))
 

In [36]:
Profile.clear()
@profile dKdθi!(Kgrad, gp_fit.k, gp_fit.X, gp_fit.data,3);
Profile.print()

1    ./fastmath.jl:0; mul_fast(::Float64, ::Float64, ::...
9643 ./task.jl:360; (::IJulia.##25#31)()
 9643 .../IJulia/src/IJulia.jl:143; eventloop(::ZMQ.Socket)
  9643 ...rc/execute_request.jl:170; execute_request(::ZMQ.Socket, ::...
   9643 ./loading.jl:380; include_string(::String, ::String)
    9643 ./<missing>:?; anonymous
     9643 ./profile.jl:16; macro expansion;
      9643 ...rnels/sum_kernel.jl:99; dKdθi!(::Array{Float64,2}, ::...
       2    ...kernels/periodic.jl:47; dKdθi!(::Array{Float64,2}, :...
       9641 ...kernels/periodic.jl:65; dKdθi!(::Array{Float64,2}, :...
        2    ./simdloop.jl:68; macro expansion
        1    ./simdloop.jl:71; macro expansion
        9638 ./simdloop.jl:73; macro expansion
         9507 ...ernels/periodic.jl:66; macro expansion
          2310 ./fastmath.jl:0; mul_fast(::Float64, ::Float...
          6266 ./fastmath.jl:149; mul_fast(::Float64, ::Float...
           2005 ./fastmath.jl:0; mul_fast(::Float64, ::Floa...
           3238 ./fastmath.

In [112]:
function grad_stack!(stack::AbstractArray, rq::RQIso, X::Matrix{Float64}, data::GaussianProcesses.IsotropicData)
    nobsv = size(X,2)
    R = GaussianProcesses.distance(rq, X, data)
    
    for j in 1:nobsv, i in 1:j
        # Check these derivatives!
        @inbounds stack[i,j,1] = rq.σ2*((R[i,j])/rq.ℓ2)*(1.0+(R[i,j])/(2*rq.α*rq.ℓ2))^(-rq.α-1.0)  # dK_d(log ℓ)dK_dℓ
        @inbounds stack[i,j,2] = 2.0*rq.σ2*(1+(R[i,j])/(2*rq.α*rq.ℓ2))^(-rq.α) # dK_d(log σ)

        part = (1.0+R[i,j]/(2*rq.α*rq.ℓ2))
        @inbounds stack[i,j,3] = rq.σ2*part^(-rq.α)*((R[i,j])/(2*rq.ℓ2*part)-rq.α*log(part))*rq.α  # dK_d(log α)
        
        @inbounds stack[j,i,1] = stack[i,j,1]
        @inbounds stack[j,i,3] = stack[i,j,3]
        @inbounds stack[j,i,2] = stack[i,j,2]
    end
    return stack
end


grad_stack! (generic function with 1 method)

In [113]:
Ksmall = Array(Float64, gp_small.nobsv, gp_small.nobsv, GaussianProcesses.num_params(_rq))
Kgrads = Array(Float64, gp_fit.nobsv, gp_fit.nobsv, GaussianProcesses.num_params(_rq))
grad_stack!(Ksmall, _rq, gp_small.X, gp_small.data.kdatas[3])
@assert isapprox(Ksmall, prev_grad_stack)
@time grad_stack!(Kgrads, _rq, gp_fit.X, gp_fit.data.kdatas[3]);

  2.669774 seconds (6 allocations: 122.071 MB, 1.02% gc time)


In [114]:
function grad_stack!(stack::AbstractArray, rq::RQIso, X::Matrix{Float64}, data::GaussianProcesses.IsotropicData)
    nobsv = size(X,2)
    R = GaussianProcesses.distance(rq, X, data)
    # Check these derivatives!
    @inbounds for j in 1:nobsv, i in 1:j
        
        stack[i,j,1] = rq.σ2*((R[i,j])/rq.ℓ2)*(1.0+(R[i,j])/(2*rq.α*rq.ℓ2))^(-rq.α-1.0)  # dK_d(log ℓ)dK_dℓ
        stack[j,i,1] = stack[i,j,1]
    end
    @inbounds for j in 1:nobsv, i in 1:j
        stack[i,j,2] = 2.0*rq.σ2*(1+(R[i,j])/(2*rq.α*rq.ℓ2))^(-rq.α) # dK_d(log σ)
        stack[j,i,2] = stack[i,j,2]
        
    end
    @inbounds for j in 1:nobsv, i in 1:j
        part = (1.0+R[i,j]/(2*rq.α*rq.ℓ2))
        stack[i,j,3] = rq.σ2*part^(-rq.α)*((R[i,j])/(2*rq.ℓ2*part)-rq.α*log(part))*rq.α  # dK_d(log α)
        stack[j,i,3] = stack[i,j,3]
    end   
    return stack
end

grad_stack! (generic function with 1 method)

In [115]:
Ksmall = Array(Float64, gp_small.nobsv, gp_small.nobsv, GaussianProcesses.num_params(_rq))
Kgrads = Array(Float64, gp_fit.nobsv, gp_fit.nobsv, GaussianProcesses.num_params(_rq))
grad_stack!(Ksmall, _rq, gp_small.X, gp_small.data.kdatas[3])
@assert isapprox(Ksmall, prev_grad_stack)
@time grad_stack!(Kgrads, _rq, gp_fit.X, gp_fit.data.kdatas[3]);

  2.598202 seconds (6 allocations: 122.071 MB, 1.27% gc time)


In [116]:
function grad_stack!(stack::AbstractArray, rq::RQIso, X::Matrix{Float64}, data::GaussianProcesses.IsotropicData)
    nobsv = size(X,2)
    R = GaussianProcesses.distance!(rq, X, data)
    # Check these derivatives!
    @inbounds @fastmath for j in 1:nobsv
        for i in 1:j        
            stack[i,j,1] = rq.σ2*((R[i,j])/rq.ℓ2)*(1.0+(R[i,j])/(2*rq.α*rq.ℓ2))^(-rq.α-1.0)  # dK_d(log ℓ)dK_dℓ
            stack[j,i,1] = stack[i,j,1]
        end
    end
    @inbounds @fastmath for j in 1:nobsv
        for i in 1:j
            stack[i,j,2] = 2.0*rq.σ2*(1+(R[i,j])/(2*rq.α*rq.ℓ2))^(-rq.α) # dK_d(log σ)
            stack[j,i,2] = stack[i,j,2]
        end
    end
        
    @inbounds @fastmath for j in 1:nobsv
        for i in 1:j
            part = (1.0+R[i,j]/(2*rq.α*rq.ℓ2))
            stack[i,j,3] = rq.σ2*part^(-rq.α)*((R[i,j])/(2*rq.ℓ2*part)-rq.α*log(part))*rq.α  # dK_d(log α)
            stack[j,i,3] = stack[i,j,3]
        end
    end   
    return stack
end

grad_stack! (generic function with 1 method)

In [117]:
Ksmall = Array(Float64, gp_small.nobsv, gp_small.nobsv, GaussianProcesses.num_params(_rq))
Kgrads = Array(Float64, gp_fit.nobsv, gp_fit.nobsv, GaussianProcesses.num_params(_rq))
grad_stack!(Ksmall, _rq, gp_small.X, gp_small.data.kdatas[3])
@assert isapprox(Ksmall, prev_grad_stack)
@time grad_stack!(Kgrads, _rq, gp_fit.X, gp_fit.data.kdatas[3]);

  2.422838 seconds (4 allocations: 160 bytes)


In [118]:
function grad_stack!(stack::AbstractArray, rq::RQIso, X::Matrix{Float64}, data::GaussianProcesses.IsotropicData)
    nobsv = size(X,2)
    R = GaussianProcesses.distance!(rq, X, data)

    # Check these derivatives!
    @inbounds @fastmath for j in 1:nobsv
        @simd for i in 1:j        
            stack[i,j,1] = rq.σ2*((R[i,j])/rq.ℓ2)*(1.0+(R[i,j])/(2*rq.α*rq.ℓ2))^(-rq.α-1.0)  # dK_d(log ℓ)dK_dℓ
            stack[j,i,1] = stack[i,j,1]
        end
    end
    @inbounds @fastmath for j in 1:nobsv
        @simd for i in 1:j
            stack[i,j,2] = 2.0*rq.σ2*(1+(R[i,j])/(2*rq.α*rq.ℓ2))^(-rq.α) # dK_d(log σ)
            stack[j,i,2] = stack[i,j,2]
        end
    end
        
    @inbounds @fastmath for j in 1:nobsv
        @simd for i in 1:j
            part = (1.0+R[i,j]/(2*rq.α*rq.ℓ2))
            stack[i,j,3] = rq.σ2*part^(-rq.α)*((R[i,j])/(2*rq.ℓ2*part)-rq.α*log(part))*rq.α  # dK_d(log α)
            stack[j,i,3] = stack[i,j,3]
        end
    end   
    return stack
end

grad_stack! (generic function with 1 method)

In [119]:
Ksmall = Array(Float64, gp_small.nobsv, gp_small.nobsv, GaussianProcesses.num_params(_rq))
Kgrads = Array(Float64, gp_fit.nobsv, gp_fit.nobsv, GaussianProcesses.num_params(_rq))
grad_stack!(Ksmall, _rq, gp_small.X, gp_small.data.kdatas[3])
@assert isapprox(Ksmall, prev_grad_stack)
@time grad_stack!(Kgrads, _rq, gp_fit.X, gp_fit.data.kdatas[3]);

  2.394021 seconds (4 allocations: 160 bytes)


In [120]:
function grad_stack!(stack::AbstractArray, rq::RQIso, X::Matrix{Float64}, data::GaussianProcesses.IsotropicData)
    nobsv = size(X,2)
    R = GaussianProcesses.distance!(rq, X, data)
    σ2 = rq.σ2
    α = rq.α
    ℓ2 = rq.ℓ2
    # Check these derivatives!
    @inbounds @fastmath for j in 1:nobsv
        @simd for i in 1:j        
            stack[i,j,1] = σ2*((R[i,j])/ℓ2)*(1.0+(R[i,j])/(2*α*ℓ2))^(-α-1.0)  # dK_d(log ℓ)dK_dℓ
            stack[j,i,1] = stack[i,j,1]
        end
    end
    @inbounds @fastmath for j in 1:nobsv
        @simd for i in 1:j
            stack[i,j,2] = 2.0*σ2*(1+(R[i,j])/(2*α*ℓ2))^(-α) # dK_d(log σ)
            stack[j,i,2] = stack[i,j,2]
        end
    end
        
    @inbounds @fastmath for j in 1:nobsv
        @simd for i in 1:j
            part = (1.0+R[i,j]/(2*α*ℓ2))
            stack[i,j,3] = σ2*part^(-α)*((R[i,j])/(2*ℓ2*part)-α*log(part))*α  # dK_d(log α)
            stack[j,i,3] = stack[i,j,3]
        end
    end   
    return stack
end

grad_stack! (generic function with 1 method)

In [121]:
Ksmall = Array(Float64, gp_small.nobsv, gp_small.nobsv, GaussianProcesses.num_params(_rq))
Kgrads = Array(Float64, gp_fit.nobsv, gp_fit.nobsv, GaussianProcesses.num_params(_rq))
grad_stack!(Ksmall, _rq, gp_small.X, gp_small.data.kdatas[3])
@assert isapprox(Ksmall, prev_grad_stack)
@time grad_stack!(Kgrads, _rq, gp_fit.X, gp_fit.data.kdatas[3]);

  2.369990 seconds (4 allocations: 160 bytes)


In [122]:
function grad_stack!(stack::AbstractArray, rq::RQIso, X::Matrix{Float64}, data::GaussianProcesses.IsotropicData)
    nobsv = size(X,2)
    R = GaussianProcesses.distance!(rq, X, data)
    σ2 = rq.σ2
    α = rq.α
    ℓ2 = rq.ℓ2
    α2ℓ2 = 2.0*α*ℓ2
    # Check these derivatives!
    @inbounds @fastmath for j in 1:nobsv
        @simd for i in 1:j        
            stack[i,j,1] = σ2*(R[i,j]/ℓ2)*(1.0+R[i,j]/α2ℓ2)^(-α-1.0)  # dK_d(log ℓ)dK_dℓ
            stack[j,i,1] = stack[i,j,1]
        end
    end
    @inbounds @fastmath for j in 1:nobsv
        @simd for i in 1:j
            stack[i,j,2] = 2.0*σ2*(1.0+R[i,j]/α2ℓ2)^(-α) # dK_d(log σ)
            stack[j,i,2] = stack[i,j,2]
        end
    end
        
    @inbounds @fastmath for j in 1:nobsv
        @simd for i in 1:j
            part = (1.0+R[i,j]/α2ℓ2)
            stack[i,j,3] = σ2*part^(-α)*((R[i,j])/(2.0*ℓ2*part)-α*log(part))*α  # dK_d(log α)
            stack[j,i,3] = stack[i,j,3]
        end
    end   
    return stack
end

grad_stack! (generic function with 1 method)

In [123]:
Ksmall = Array(Float64, gp_small.nobsv, gp_small.nobsv, GaussianProcesses.num_params(_rq))
Kgrads = Array(Float64, gp_fit.nobsv, gp_fit.nobsv, GaussianProcesses.num_params(_rq))
grad_stack!(Ksmall, _rq, gp_small.X, gp_small.data.kdatas[3])
@assert isapprox(Ksmall, prev_grad_stack)
@time grad_stack!(Kgrads, _rq, gp_fit.X, gp_fit.data.kdatas[3]);

  2.385509 seconds (4 allocations: 160 bytes)


## Optimize update_mll_and_dmll

In [126]:
GaussianProcesses.update_mll_and_dmll!(gp_small);
# @time GaussianProcesses.update_mll_and_dmll!(gp_fit);
prev_dmLL=gp_small.dmLL

16-element Array{Float64,1}:
      2.05225e8
     -4.10579  
     -1.83856e7
      2.26609e6
      7.07995e8
     -1.87513e7
      1.76462e6
     -3.97544e7
 -26706.3      
  34353.6      
   1522.72     
   -188.466    
    171.744    
  -4388.32     
   -359.407    
   1760.4      

In [165]:
function update_mll_and_dmll!(gp::GP; noise::Bool=true, mean::Bool=true, kern::Bool=true)
    update_mll!!(gp)
    gp.dmLL = Array(Float64, noise + mean*num_params(gp.m) + kern*num_params(gp.k))

    # Calculate Gradient with respect to hyperparameters

    #Derivative wrt the observation noise
    if noise
        gp.dmLL[1] = exp(2*gp.logNoise)*trace((gp.alpha*gp.alpha' - gp.cK \ eye(gp.nobsv)))
    end

    #Derivative wrt to mean hyperparameters, need to loop over as same with kernel hyperparameters
    if mean
        Mgrads = grad_stack(gp.m, gp.X)
        for i in 1:num_params(gp.m)
            gp.dmLL[i+noise] = dot(Mgrads[:,i],gp.alpha)
        end
    end

    # Derivative of marginal log-likelihood with respect to kernel hyperparameters
    if kern
        Kgrads = grad_stack(gp.k, gp.X, gp.data)   # [dK/dθᵢ]
        for i in 1:num_params(gp.k)
            gp.dmLL[i+mean*num_params(gp.m)+noise] = trace((gp.alpha*gp.alpha' - gp.cK \ eye(gp.nobsv))*Kgrads[:,:,i])/2
        end
    end
end

update_mll_and_dmll! (generic function with 2 methods)

In [166]:
update_mll_and_dmll!(gp_small);
@assert isapprox(prev_dmLL,gp_small.dmLL)
@time GaussianProcesses.update_mll_and_dmll!(gp_fit);

 65.075510 seconds (206.38 k allocations: 13.594 GB, 13.20% gc time)


In [193]:
function update_mll_and_dmll!(gp::GP,
    Kgrads::Array{Float64,3}
    ; 
    noise::Bool=true, # include gradient component for the logNoise term
    mean::Bool=true, # include gradient components for the mean parameters
    kern::Bool=true, # include gradient components for the spatial kernel parameters
    )
    update_mll!!(gp)
    n_mean_params = num_params(gp.m)
    n_kern_params = num_params(gp.k)
    gp.dmLL = Array(Float64, noise + mean*n_mean_params + kern*n_kern_params)
    logNoise = gp.logNoise
    α = gp.alpha
    ααinvcKI = α*α' - gp.cK \ eye(gp.nobsv)
    i=1
    if noise
        gp.dmLL[i] = exp(2.0*logNoise)*trace(ααinvcKI)
        i+=1
    end
    if mean
        Mgrads = grad_stack(gp.m, gp.X)
        for j in 1:n_mean_params
            gp.dmLL[i] = dot(Mgrads[:,j],gp.alpha)
            i += 1
        end
    end
    if kern
#         grad_stack!(Kgrads, gp.k, gp.X, gp.data)
        GaussianProcesses.grad_stack!(Kgrads, gp.k, gp.X, gp.data)
        for j in 1:n_kern_params
            gp.dmLL[i] = trace(Kgrads[:,:,j] * ααinvcKI) / 2.0
            i += 1
        end
    end
end

update_mll_and_dmll! (generic function with 2 methods)

In [194]:
Ksmall = Array(Float64, gp_small.nobsv, gp_small.nobsv, GaussianProcesses.num_params(gp_small.k))
Kgrads = Array(Float64, gp_fit.nobsv, gp_fit.nobsv, GaussianProcesses.num_params(gp_fit.k))
update_mll_and_dmll!(gp_small, Ksmall);
@time update_mll_and_dmll!(gp_fit, Kgrads);
@assert isapprox(prev_dmLL,gp_small.dmLL)

in mine
in mine
 37.318876 seconds (56.41 k allocations: 4.293 GB, 2.14% gc time)


In [201]:
function update_mll_and_dmll!(gp::GP,
    Kgrads::Array{Float64,3}
    ; 
    noise::Bool=true, # include gradient component for the logNoise term
    mean::Bool=true, # include gradient components for the mean parameters
    kern::Bool=true, # include gradient components for the spatial kernel parameters
    )
    update_mll!!(gp)
    n_mean_params = num_params(gp.m)
    n_kern_params = num_params(gp.k)
    gp.dmLL = Array(Float64, noise + mean*n_mean_params + kern*n_kern_params)
    logNoise = gp.logNoise
    α = gp.alpha
    ααinvcKI = α*α' - gp.cK \ eye(gp.nobsv)
    i=1
    if noise
        gp.dmLL[i] = exp(2.0*logNoise)*trace(ααinvcKI)
        i+=1
    end
    if mean
        Mgrads = grad_stack(gp.m, gp.X)
        for j in 1:n_mean_params
            gp.dmLL[i] = dot(Mgrads[:,j],gp.alpha)
            i += 1
        end
    end
    if kern
#         grad_stack!(Kgrads, gp.k, gp.X, gp.data)
        GaussianProcesses.grad_stack!(Kgrads, gp.k, gp.X, gp.data)
        for j in 1:n_kern_params
            gp.dmLL[i] = sum(Kgrads[:,:,j] .* ααinvcKI) / 2.0
            i += 1
        end
    end
end

update_mll_and_dmll! (generic function with 2 methods)

In [202]:
Ksmall = Array(Float64, gp_small.nobsv, gp_small.nobsv, GaussianProcesses.num_params(gp_small.k))
Kgrads = Array(Float64, gp_fit.nobsv, gp_fit.nobsv, GaussianProcesses.num_params(gp_fit.k))
update_mll_and_dmll!(gp_small, Ksmall);
@assert isapprox(prev_dmLL,gp_small.dmLL)

LoadError: LoadError: AssertionError: isapprox(prev_dmLL,gp_small.dmLL)
while loading In[202], in expression starting on line 4

In [203]:
inter_dmLL = gp_small.dmLL
@time update_mll_and_dmll!(gp_fit, Kgrads);

 19.840617 seconds (56.42 k allocations: 4.293 GB, 5.01% gc time)


In [218]:
function update_mll_and_dmll!(gp::GP,
    Kgrads::Array{Float64,3}
    ; 
    noise::Bool=true, # include gradient component for the logNoise term
    mean::Bool=true, # include gradient components for the mean parameters
    kern::Bool=true, # include gradient components for the spatial kernel parameters
    )
    println("in mine")
    update_mll!!(gp)
    n_mean_params = num_params(gp.m)
    n_kern_params = num_params(gp.k)
    gp.dmLL = Array(Float64, noise + mean*n_mean_params + kern*n_kern_params)
    logNoise = gp.logNoise
    α = gp.alpha
    ααinvcKI = α*α' - gp.cK \ eye(gp.nobsv)
    i=1
    if noise
        gp.dmLL[i] = exp(2.0*logNoise)*trace(ααinvcKI)
        i+=1
    end
    if mean
        Mgrads = grad_stack(gp.m, gp.X)
        for j in 1:n_mean_params
            gp.dmLL[i] = dot(Mgrads[:,j],gp.alpha)
            i += 1
        end
    end
    if kern
#         grad_stack!(Kgrads, gp.k, gp.X, gp.data)
        GaussianProcesses.grad_stack!(Kgrads, gp.k, gp.X, gp.data)
        for j in 1:n_kern_params
            dmLL = 0.0
            
            for k in 1:gp.nobsv
                @inbounds dmLL += Kgrads[k,k,j] * ααinvcKI[k,k] / 2.0
                @inbounds @simd for l in 1:k-1
                    dmLL += Kgrads[l,k,j] * ααinvcKI[l,k]
                end
            end
            gp.dmLL[i] = dmLL
            i += 1
        end
    end
end

update_mll_and_dmll! (generic function with 2 methods)

In [219]:
Ksmall = Array(Float64, gp_small.nobsv, gp_small.nobsv, GaussianProcesses.num_params(gp_small.k))
Kgrads = Array(Float64, gp_fit.nobsv, gp_fit.nobsv, GaussianProcesses.num_params(gp_fit.k))
update_mll_and_dmll!(gp_small, Ksmall);
@assert isapprox(prev_dmLL,gp_small.dmLL)

in mine


LoadError: LoadError: AssertionError: isapprox(prev_dmLL,gp_small.dmLL)
while loading In[219], in expression starting on line 4

In [220]:
@time update_mll_and_dmll!(gp_fit, Kgrads);

in mine
 31.976214 seconds (839.09 M allocations: 13.458 GB, 4.77% gc time)


In [258]:
function update_mll_and_dmll!(gp::GP,
    Kgrads::Array{Float64,3}
    ; 
    noise::Bool=true, # include gradient component for the logNoise term
    mean::Bool=true, # include gradient components for the mean parameters
    kern::Bool=true, # include gradient components for the spatial kernel parameters
    )
    update_mll!!(gp)
    n_mean_params = num_params(gp.m)
    n_kern_params = num_params(gp.k)
    gp.dmLL = Array(Float64, noise + mean*n_mean_params + kern*n_kern_params)
    logNoise = gp.logNoise
    α = gp.alpha
    ααinvcKI = α*α' - gp.cK \ eye(gp.nobsv)
    i=1
    if noise
        gp.dmLL[i] = exp(2.0*logNoise)*trace(ααinvcKI)
        i+=1
    end
    if mean
        Mgrads = grad_stack(gp.m, gp.X)
        for j in 1:n_mean_params
            gp.dmLL[i] = dot(Mgrads[:,j],gp.alpha)
            i += 1
        end
    end
    if kern
#         grad_stack!(Kgrads, gp.k, gp.X, gp.data)
        GaussianProcesses.grad_stack!(Kgrads, gp.k, gp.X, gp.data)
        broadcast!(.*, Kgrads, Kgrads, ααinvcKI)
        gp.dmLL[i:i+n_kern_params-1] = vec(sum(Kgrads,[1,2]))/2
        i+=n_kern_params
    end
end

update_mll_and_dmll! (generic function with 2 methods)

In [259]:
Ksmall = Array(Float64, gp_small.nobsv, gp_small.nobsv, GaussianProcesses.num_params(gp_small.k))
Kgrads = Array(Float64, gp_fit.nobsv, gp_fit.nobsv, GaussianProcesses.num_params(gp_fit.k))
update_mll_and_dmll!(gp_small, Ksmall);
@assert isapprox(inter_dmLL,gp_small.dmLL)

In [260]:
@time update_mll_and_dmll!(gp_fit, Kgrads);

 17.423403 seconds (56.13 k allocations: 978.247 MB, 3.04% gc time)


In [252]:
@time update_mll_and_dmll!(gp_fit, Kgrads, noise=false, mean=false, kern=false);

  4.192510 seconds (47.25 k allocations: 489.425 MB, 4.83% gc time)


In [253]:
@time update_mll_and_dmll!(gp_fit, Kgrads, noise=true, mean=false, kern=false);

  4.054616 seconds (37.01 k allocations: 489.000 MB, 1.91% gc time)


In [254]:
@time update_mll_and_dmll!(gp_fit, Kgrads, noise=false, mean=true, kern=false);

  4.403914 seconds (55.99 k allocations: 489.961 MB, 5.35% gc time)


In [256]:
@time update_mll_and_dmll!(gp_fit, Kgrads, noise=false, mean=false, kern=true);

 17.900951 seconds (37.14 k allocations: 977.286 MB, 2.36% gc time)


# Update mLL

That should only take $4000^2 \times 64\mathrm{bit} = 128\mathrm{MB}$

In [6]:
@time GaussianProcesses.update_mll!(gp_fit);
prev_mLL=gp_fit.mLL

  3.539167 seconds (22.03 k allocations: 977.022 MB, 17.57% gc time)


-1.4530769893150377e10

In [63]:
# modification of update_mll! that reuses existing matrices to avoid
# unnecessary memory allocations, which speeds things up significantly
function update_mll!!(gp::GP)
    Σbuffer = gp.cK.mat
    μ = mean(gp.m,gp.X)
    GaussianProcesses.cov!(Σbuffer, gp.k, gp.X, gp.data)
    for i in 1:gp.nobsv
        Σbuffer[i,i] += exp(2*gp.logNoise) + 1e-8
    end
    chol_buffer = gp.cK.chol.factors
    copy!(chol_buffer, Σbuffer)
    chol = cholfact!(Symmetric(chol_buffer))
    gp.cK = PDMats.PDMat(Σbuffer, chol)
    gp.alpha = gp.cK \ (gp.y - μ)
    gp.mLL = -dot((gp.y - μ),gp.alpha)/2.0 - logdet(gp.cK)/2.0 - gp.nobsv*log(2π)/2.0 # Marginal log-likelihood
end

update_mll!! (generic function with 1 method)

In [8]:
@time update_mll!!(gp_fit);
@assert isapprox(gp_fit.mLL, prev_mLL)

  2.552592 seconds (64.43 k allocations: 2.104 MB)


In [9]:
Σbuffer = zeros(gp_fit.nobsv, gp_fit.nobsv)
GaussianProcesses.cov!(Σbuffer, gp_fit.k, gp_fit.X, gp_fit.data);
@time GaussianProcesses.cov!(Σbuffer, gp_fit.k, gp_fit.X, gp_fit.data);

  2.248447 seconds (9 allocations: 320 bytes)


In [10]:
Σbuffer = zeros(gp_fit.nobsv, gp_fit.nobsv)
GaussianProcesses.addcov!(Σbuffer, gp_fit.k.kerns[3], gp_fit.X, gp_fit.data.kdatas[3]);
@time GaussianProcesses.addcov!(Σbuffer, gp_fit.k.kerns[3], gp_fit.X, gp_fit.data.kdatas[3]);

  0.657412 seconds (4 allocations: 160 bytes)


A huge amount of time is spent constructing the covariance matrix! What a waste.

In [ ]:
prevK=GaussianProcesses.cov(gp_fit.k, gp_fit.X, gp_fit.data);
@time GaussianProcesses.cov(gp_fit.k, gp_fit.X, gp_fit.data);